In [1]:
import sys
sys.path.append('..')
%env CUDA_VISIBLE_DEVICES=0


env: CUDA_VISIBLE_DEVICES=0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn


In [3]:
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [4]:
from utils import data_utils

data = data_utils.POWER()

dim = data.n_dims
num_layers = 3


In [5]:
X_train_tensor = torch.from_numpy(data.trn.x)
X_test_tensor = torch.from_numpy(data.tst.x)

In [6]:
# def init_weights(m):
#     if 'Linear' in str(type(m)):
#         torch.nn.init.xavier_uniform_(m.weight)
#         m.bias.data.fill_(0.01)


In [7]:
from torch.distributions import MultivariateNormal

prior = MultivariateNormal(torch.zeros(data.n_dims).to(device),
                           torch.eye(data.n_dims).to(device))


In [8]:
from src.mrartemev_nflib.flows import NormalizingFlowModel, InvertiblePermutation
from src.mrartemev_nflib.flows import ActNorm, NSF_AR, Invertible1x1Conv
from src.mrartemev_nflib.nn import MLP

flows = []
for _ in range(num_layers):
    flows.append(ActNorm(dim=data.n_dims))
    flows.append(Invertible1x1Conv(dim=data.n_dims))
    flows.append(NSF_AR(dim=data.n_dims, K=8, B=3, hidden_features=16, base_network=MLP))
model = NormalizingFlowModel(prior, flows).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [9]:
from torch.utils.data import Dataset, DataLoader, TensorDataset

dloader = DataLoader(TensorDataset(X_train_tensor), batch_size=2**14,
                     shuffle=True, drop_last=True, num_workers=4)
test_dloader = DataLoader(TensorDataset(X_test_tensor), batch_size=2**14,
                          shuffle=True, drop_last=True, num_workers=4)


In [10]:
from IPython.display import clear_output
from tqdm import tqdm


epoches = 10000
plot_freq = 20

epoch_losses = []
train_losses = []

In [ ]:
for epoch in tqdm(range(epoches), position=0):
    for ind, batch in enumerate(dloader):
        model.train()
        
        # fit
        optimizer.zero_grad()
        logp_x = model.log_prob(batch[0].to(device))
        loss = -torch.mean(logp_x)
        loss.backward()
        
        # clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
                
        # step
        optimizer.step()

        # log
        train_losses.append(-loss.item())
        
    with torch.no_grad():
        model.eval()
        logp_x = np.mean([torch.mean(model.log_prob(batch[0].to(device))).item() for batch in test_dloader])
        epoch_losses.append([np.mean(train_losses), logp_x])
    
    if epoch % plot_freq == 0 and epoch != 0:
        clear_output(True)
        plt.figure(figsize=(15, 5))
        val_losses = np.array(epoch_losses)
        plt.plot(val_losses[:, 0], label='train_mean_loss')
        plt.plot(val_losses[:, 1], label='val_mean_loss')
        plt.ylim(-10, 10)
        plt.grid()
        plt.legend()
        plt.show()
        

  0%|          | 10/10000 [08:01<132:55:09, 47.90s/it]

In [ ]:
os.makedirs('../models/POWER/SPLINE-AR_3/', exist_ok=True)

torch.save(model.state_dict(), '../models/POWER/SPLINE-AR_3/model.state_dict')
torch.save(optimizer.state_dict(), '../models/POWER/SPLINE-AR_3/optimizer.state_dict')

In [ ]:

model.load_state_dict(torch.load('../models/POWER/SPLINE-AR_3/model.state_dict'))

Model eval + move model to cpu

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

from src.nf import CalibratedModel, neg_log_likelihood
from src.nf.classifiers import train_catboost_clf
from scipy.special import logsumexp


In [ ]:
model.eval()
model.sample_n = lambda x: model.sample(x)

model.cpu()
model.prior = MultivariateNormal(torch.zeros(data.n_dims),
                                 torch.eye(data.n_dims))


print('Model train LL ', torch.mean(model.log_prob(X_train_tensor)).item())
print('Model test LL ', torch.mean(model.log_prob(X_test_tensor)).item())


In [ ]:
n = 100000

clf_ds_train = np.row_stack([
    np.column_stack([X_train_tensor.detach().numpy()[:n], np.ones(n).reshape(-1, 1)]),
    np.column_stack([model.sample(n).detach().numpy()[:n], np.zeros(n).reshape(-1, 1)])
]).astype(np.float32)

clf_ds_test = np.row_stack([
    np.column_stack([X_test_tensor.detach().numpy()[:n], np.ones(n).reshape(-1, 1)]),
    np.column_stack([model.sample(n).detach().numpy()[:n], np.zeros(n).reshape(-1, 1)])
]).astype(np.float32)

In [ ]:
clf = CatBoostClassifier(
    1000, eval_metric='AUC',
    metric_period=200,
).fit(
    clf_ds_train[:, :-1], clf_ds_train[:, -1],
    eval_set=(clf_ds_test[:, :-1], clf_ds_test[:, -1])
)

In [ ]:
calibrated_model = CalibratedModel(clf, model, logit=True)

samples = model.sample(10000).detach().cpu().numpy()
clf_preds = clf.predict(samples, prediction_type='RawFormulaVal')
calibration_constant = logsumexp(clf_preds) - np.log(len(clf_preds))

-neg_log_likelihood(model, X_test_tensor), \
-neg_log_likelihood(calibrated_model, X_test_tensor) - calibration_constant,

In [ ]:
clf2000 = CatBoostClassifier(
    2000, eval_metric='AUC',
    metric_period=500,
).fit(
    clf_ds_train[:, :-1], clf_ds_train[:, -1],
    eval_set=(clf_ds_test[:, :-1], clf_ds_test[:, -1])
)

samples = model.sample(10000).detach().cpu().numpy()
clf_preds = clf2000.predict(samples, prediction_type='RawFormulaVal')
calibration_constant = logsumexp(clf_preds) - np.log(len(clf_preds))

calibrated_model2000 = CalibratedModel(clf2000, model, logit=True)

-neg_log_likelihood(model, X_test_tensor), \
-neg_log_likelihood(calibrated_model2000, X_test_tensor)